#### Import Modules,  Avoid Sleep Mode,  Reload Packages

In [1]:
import pandas as pd
import sqlite3
import pickle
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from ordered_set import OrderedSet

import importlib
import utils.db_funcs
import utils.df_funcs
import utils.fmp_funcs
import utils.graph_db_funcs

importlib.reload(utils.db_funcs)
importlib.reload(utils.df_funcs) 
importlib.reload(utils.fmp_funcs) 
importlib.reload(utils.graph_db_funcs)


from utils import db_funcs as db
from utils.df_funcs import *
from utils.fmp_funcs import *
from utils.graph_db_funcs import *

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


import subprocess
import atexit
caffeinate_process = subprocess.Popen(["caffeinate", "-i"])
print("Caffeinate started to prevent sleep mode.")
def stop_caffeinate():
    caffeinate_process.terminate()
    print("Caffeinate stopped. Sleep mode enabled.")
atexit.register(stop_caffeinate)

neo4j+s://461e473a.databases.neo4j.io
neo4j+s://461e473a.databases.neo4j.io
Caffeinate started to prevent sleep mode.


<function __main__.stop_caffeinate()>

#### Read all data from SQLITE, Make FX Conversion to EUR

In [2]:
print("Run initiated")

conn = sqlite3.connect(db.db_file)
forex_rates = pd.read_sql_query("SELECT * FROM exchange_rates", conn).sort_values(by=['date'], ascending=False).reset_index(drop=True)   
stock_info  = pd.read_sql_query("SELECT * FROM stock_info", conn)
stock_prices = pd.read_sql_query("SELECT * FROM hist_prices", conn).sort_values(by=['date'], ascending=False).reset_index(drop=True)   
stock_mcaps = pd.read_sql_query("SELECT * FROM hist_mcaps", conn).sort_values(by=['date'], ascending=False).reset_index(drop=True)
print("SQLITE Data loaded")

forex_rates['EUREUR'] = 1
stock_info, long_stock_info = transform_stock_info(stock_info, forex_rates, country_codes, stock_subset=16000)
stock_prices = transpose_df(stock_prices, values='close')
stock_mcaps = transpose_df(stock_mcaps, values='mcap')
#all_symbols_16000 = stock_info.symbol.to_list()
#with open("files/all_symbols_16000.pkl", "wb") as file:
#    pickle.dump(all_symbols_16000, file)
print("Data transformed")


stock_ccy_dict = dict(zip(stock_info['symbol'], stock_info['currency']))
mapping_dict = {key: 'EUR' + value for key, value in stock_ccy_dict.items()}

price_columns = OrderedSet(stock_prices.columns) & OrderedSet(stock_info.symbol)
mcaps_columns = OrderedSet(stock_mcaps.columns) & OrderedSet(stock_info.symbol)

stock_prices = stock_prices[['date'] + list(price_columns)]
stock_mcaps = stock_mcaps[['date'] + list(mcaps_columns)]

stock_prices_eur = fx_converter(stock_prices, forex_rates, mapping_dict)
stock_mcaps_eur = fx_converter(stock_mcaps, forex_rates, mapping_dict)
print("Prices converted")


Run initiated
SQLITE Data loaded


#### Adding Features, Calculate Returns, Trimming Prices, Save Clean Date to PICKLE

In [4]:

pc_final_inter, mc_final_inter = process_data(stock_prices_eur, stock_mcaps_eur, gap_size=50)
#pc_final_inter, mc_final_inter = process_data(stock_prices, stock_mcaps, gap_size=50)
prices, mcaps = align_prices_and_mcaps(pc_final_inter, mc_final_inter)
_, _, _ = make_comparison_tables(prices, mcaps)
prices_trimmed, mcaps_trimmed, removed_stocks = trim_stocks_with_little_data(prices, mcaps, days=250)

data_points = calculate_number_of_price_data_points(prices_trimmed)
annual_returns = calculate_annualized_returns(prices_trimmed)
annual_vols = calculate_annualized_volatilities(prices_trimmed)
mcap_dict = dict(zip(stock_info['symbol'], stock_info['market_cap_euro']))
trade_volume_dict = dict(zip(stock_info['symbol'], stock_info['avg_trade_vol_euro']))

return_bins = make_bins(annual_returns)
vol_bins = make_bins(annual_vols)
mcap_bins = make_bins(mcap_dict)
trade_vol_bins = make_bins(trade_volume_dict)

stock_info["data_points"] = stock_info["symbol"].map(data_points)
stock_info["return_coeff"] = stock_info["symbol"].map(annual_returns)
stock_info["volatility_coeff"] = stock_info["symbol"].map(annual_vols)

stock_info['description'] = stock_info.apply( lambda row: f"Name: {row['name']}, Symbol: {row['symbol']}, Country: {row['country']}, Description: {row['description']}",axis=1)
stock_info["return"] = stock_info["symbol"].map(return_bins)
stock_info["volatility"] = stock_info["symbol"].map(vol_bins)
stock_info["market_capitalization"] = stock_info["symbol"].map(mcap_bins)
stock_info["average_trading_volume"] = stock_info["symbol"].map(trade_vol_bins)

stock_info_final = stock_info[stock_info['symbol'].isin(list(prices_trimmed.columns)[1:])]

# with open("files/stock_info_final.pkl", "wb") as file:
#    pickle.dump(stock_info_final, file)

# with open("files/prices_trimmed.pkl", "wb") as file:
#    pickle.dump(prices_trimmed, file)

# with open("files/mcaps_trimmed.pkl", "wb") as file:
#    pickle.dump(mcaps_trimmed, file)

Number of misaligned Stocks:  13876
Number of Stocks for which NaN were introduced:  603
Number of Stocks dropped in both Dataframes:  370 

Number of misaligned Stocks:  13506
Number of Stocks for which NaN were introduced:  0
Number of Stocks dropped in both Dataframes:  0 

Stock number before trimming:  13507 13507
Stock number after trimming:  13358 13358


In [2]:
#stock_info_final.head(5)
#prices_trimmed.iloc[:,:20] .head(100)

#prices_trimmed[prices_trimmed['date']>='2019-01-01'].iloc[:,:10].head(1000)
# annual_returns

# max_value = min(data_points.values())

# keys_with_max_value = [key for key, value in data_points.items() if value == max_value]
# keys_with_max_value

# with open('../files/prices_trimmed.pkl', 'rb') as file:
#     prices = pickle.load(file)